In [1]:
import numpy as np
import pandas as pd
import json

# Setup

In [2]:
with open('../../config/config.json', 'r') as fh:
    params = json.load(fh)

In [3]:
zbp_detail_by_year = {}
zbp_totals_by_year = {}
hh_income_by_year = {}
total_pop_by_year = {}
retire_by_year = {}
for year in params['years']:
    shortened_year = str(year)[2:]
    detail_encoding = None
    totals_encoding = None
    if year >= 2017:
        detail_encoding = 'latin-1'
    if year >= 2018:
        totals_encoding = 'latin-1'
    zbp_detail_by_year[year] = pd.read_csv(f'../../src/data/raw/zbp_data/zbpdetail/zbp{shortened_year}detail/zbp{shortened_year}detail.txt', encoding=detail_encoding)
    zbp_totals_by_year[year] = pd.read_csv(f'../../src/data/raw/zbp_data/zbptotals/zbp{shortened_year}totals/zbp{shortened_year}totals.txt', encoding=totals_encoding)
    hh_income_by_year[year] = pd.read_csv(f'../../src/data/raw/household_data/ACSST5Y{year}.csv')
    total_pop_by_year[year] = pd.read_csv(f'../../src/data/raw/pop_age_data/ACSDP5Y{year}.csv')
    retire_by_year[year] = pd.read_csv(f'../../src/data/raw/retire_data/pop{year}.csv')

# ZBP Detail

In [4]:
def is_2dig_naics(naics_code):
    return naics_code[:2].isnumeric() and not any(char.isdigit() for char in naics_code[2:])

def process_zbp_data(data, year):
    cols = ['zip', 'naics', 'est']
    if year <= 2016:
        cols += ['n1_4']
    else:
        cols += ['n<5']
    cols += ['n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000']
    
    # filter and standardize col names
    data = data[cols]
    data = data.rename(columns={'n<5':'n1_4'})
    # filter only relavent zip codes
    data = data[data['zip'].apply(lambda x: x in params['zip_codes'])]
    # keep only 2dig naics
    data = data[data['naics'].apply(is_2dig_naics)]
    data['naics'] = data['naics'].apply(lambda x: x[:2])
    # cast est size bin cols to int, TREATING 'N'(Not available or not comparable) RECORDS AS 0
    for col in ['n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499', 'n500_999', 'n1000']:
        data[col] = data[col].apply(lambda x: x if x != 'N' else 0).astype('int64')
    # assign year variable
    data = data.assign(year=np.full(data.shape[0], year))
    
    return data

In [5]:
for year in zbp_detail_by_year:
    zbp_detail_by_year[year] = process_zbp_data(zbp_detail_by_year[year], year)
    
zbp_detail_data = pd.concat(list(zbp_detail_by_year.values()), ignore_index=True).reset_index(drop=True)
zbp_detail_data.to_csv('../../src/data/temp/processed_zbp_detail_data.csv', index=False)
zbp_detail_data.head()

,zip,naics,est,n1_4,n5_9,n10_19,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,year
0,91901,23,88,68,7,6,5,1,0,1,0,0,2012
1,91901,31,2,1,1,0,0,0,0,0,0,0,2012
2,91901,42,9,9,0,0,0,0,0,0,0,0,2012
3,91901,44,67,25,21,16,4,1,0,0,0,0,2012
4,91901,48,8,8,0,0,0,0,0,0,0,0,2012


# ZBP Totals

In [6]:
def process_zbp_totals(data, year):
    # drop naming columns
    cols = ['name', 'city', 'stabbr', 'cty_name']
    if year <= 2017:
        cols += ['empflag']
    data = data.drop(columns=cols)
    # filter only relavent zip codes
    data = data[data['zip'].apply(lambda x: x in params['zip_codes'])]
    # assign year variable
    data = data.assign(year=np.full(data.shape[0], year))
    return data

In [7]:
for year in zbp_totals_by_year:
    zbp_totals_by_year[year] = process_zbp_totals(zbp_totals_by_year[year], year)
    
zbp_totals_data = pd.concat(list(zbp_totals_by_year.values()), ignore_index=True).reset_index(drop=True)
zbp_totals_data.to_csv('../../src/data/temp/processed_zbp_totals_data.csv', index=False)
zbp_totals_data.head()

,zip,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,year
0,91901,H,4141,H,36304,H,174786,391,2012
1,91902,G,2265,G,19111,G,81569,349,2012
2,91903,S,0,G,123,G,491,13,2012
3,91905,G,19,S,0,H,748,9,2012
4,91906,D,0,D,0,D,0,27,2012


# Median Household Income Data Processing 

In [8]:
# Reformats zip column to be  5 digit zip code
def reformat_zip(x):
    return x[6:11]

# Median household incomes 250000+ reformatted to 250000 and commas removed to convert object types to ints
def reformat_income(x):
    x = x.replace('+', '')
    return int(x.replace(',', ''))

# FUNCTION FOR PROCESSING HOUSEHOLD INCOME BY INPUT YEAR
def process_hh(data, year):
    hh2012 = data.T
    hh2012.columns = hh2012.iloc[0]
    hh2012 = hh2012.drop(hh2012.index[0])
    hh2012 = hh2012.reset_index()
    hh2012 = hh2012.rename_axis(None, axis=1)
    hh2012 = hh2012.rename(columns = {'index':'zip'})
    # Filtering data by household estimates
    hh2012 = hh2012[hh2012["zip"].str.contains(r'^(?=.*Households)(?=.*Estimate)')]
    hh2012['zip'] = hh2012['zip'].apply(reformat_zip)
    hh2012 = hh2012[['zip','Median income (dollars)']]
    hh2012['year'] = year
    # Dropping missing values
    hh2012 = hh2012[hh2012['Median income (dollars)'].str.contains('-')==False]
    hh2012 = hh2012[hh2012['Median income (dollars)'].str.contains('X')==False]
    hh2012['Median income (dollars)'] = hh2012['Median income (dollars)'].apply(reformat_income)
    hh2012['zip'] = hh2012['zip'].astype('int64')
    # Keeping only relevant zip-codes
    hh2012 = hh2012[hh2012['zip'].apply(lambda x: x in params['zip_codes'])]
    return hh2012

In [9]:
#Processing the household income data for 2012-2021
for year in hh_income_by_year:
    hh_income_by_year[year] = process_hh(hh_income_by_year[year], year)
    
hh_income_data = pd.concat(list(hh_income_by_year.values()), ignore_index=True).reset_index(drop=True)
hh_income_data.to_csv('../../src/data/temp/processed_hh_income_data.csv', index=False)
hh_income_data.head()

,zip,Median income (dollars),year
0,91901,76496,2012
1,91902,86099,2012
2,91905,70000,2012
3,91906,54135,2012
4,91910,55875,2012


# Population Data Processing 

In [10]:
#Removing commas from population strings before converting to floats.
def reformat_pop(x):
    return float(x.replace(',', ''))

def float_check(x):
    return isinstance(x, float)

In [11]:
#Function for processing population data by year
def process_pop(data, year):
    pop = data.T
    pop.columns = pop.iloc[0]
    pop = pop.drop(pop.index[0])
    pop = pop.reset_index()
    pop = pop.rename_axis(None, axis=1)
    pop = pop.rename(columns = {'index':'zip'})
    #Filtering population data by estimates only
    pop = pop[pop["zip"].str.contains(r'^(?=.*Estimate)')]
    pop['zip'] = pop['zip'].apply(reformat_zip)
    pop_column = pop.rename(columns = {'\xa0\xa0\xa0\xa0Total population': 'Total population'})
    pop['Total population'] = pop_column['Total population'].iloc[:,:1]
    pop = pop[['zip','Total population']]
    pop['year'] = year
    #Converting columns to appropriate datatypes
    pop['zip'] = pop['zip'].astype('int64')
    pop['Total population'] = pop['Total population'].astype(str)
    pop['Total population'] = pop['Total population'].apply(reformat_pop)
    # Keeping only relevant zip-codes
    pop = pop[pop['zip'].apply(lambda x: x in params['zip_codes'])]
    return pop

In [12]:
#Processing all population estimate data from 2012-2021
for year in total_pop_by_year:
    total_pop_by_year[year] = process_pop(total_pop_by_year[year], year)

total_pop_data = pd.concat(list(total_pop_by_year.values()), ignore_index=True).reset_index(drop=True)
total_pop_data.to_csv('../../src/data/temp/processed_total_pop_data.csv', index=False)
total_pop_data.head()

,zip,Total population,year
0,91901,17034.0,2012
1,91902,17659.0,2012
2,91905,1088.0,2012
3,91906,3679.0,2012
4,91910,73761.0,2012


# Retirement Data Processing

In [13]:
def process_retire_data(data, year):
    
    column_names = data.columns
    # Only getting Estimates
    filtered_columns = [col for col in column_names if col[-8:] == 'Estimate' and col[6:11].isdigit()]
    data = data[filtered_columns]
    # Cleaning up so that the rows are just the zipcode
    data.columns = data.columns.str.replace('ZCTA5 ', '').str.replace('!!Estimate', '')
    # Selecting: Total population and retired aged
    selected_rows = data.loc[[1, 14, 15, 16]]
    # Define the new row names
    og_names = {
                    1: 'Total population',
                    14: '65 to 74 years',
                    15: '75 to 84 years',
                    16: '85 years and over'
                }
    data = selected_rows.rename(index=og_names)
    # Remove commas from all numbers in all columns
    data = data.map(lambda x: str(x).replace(',', ''))
    # Convert all columns to numeric type
    data = data.apply(pd.to_numeric, errors='ignore')
    # Transpose dataframe and convert zip-code col
    data = data.T
    data.index = data.index.set_names(['zip'])
    data = data.reset_index()
    data['zip'] = data['zip'].astype('int64')
    # Creating row for total retirement population
    data.insert(2, 'Total Retirement', data.iloc[:,2:].sum(axis=1))
    # Assign year variable
    data = data.assign(year=np.full(data.shape[0], year))
    # Keeping only relevant zip-codes
    data = data[data['zip'].apply(lambda x: x in params['zip_codes'])]
    
    return data

In [14]:
for year in retire_by_year:
    retire_by_year[year] = process_retire_data(retire_by_year[year], year)

retire_data = pd.concat(list(retire_by_year.values()), ignore_index=True).reset_index(drop=True)
retire_data.to_csv('../../src/data/temp/processed_retire_detail_data.csv', index=False)
retire_data.head()

,zip,Total population,Total Retirement,65 to 74 years,75 to 84 years,85 years and over,year
0,91901,17034.0,2691.0,1631.0,767.0,293.0,2012
1,91902,17659.0,3485.0,1949.0,1112.0,424.0,2012
2,91905,1088.0,101.0,70.0,19.0,12.0,2012
3,91906,3679.0,395.0,301.0,90.0,4.0,2012
4,91910,73761.0,9195.0,4666.0,3046.0,1483.0,2012
